# 🎉 Ejemplos de Nuevas Funcionalidades - BESTLIB

Este notebook muestra las **3 mejoras principales** implementadas:

1. ✅ **Soporte para DataFrames de pandas** - Especificar columnas directamente
2. ✅ **LinkedViews integrado en ReactiveMatrixLayout** - Trabajar dentro de la matriz ASCII
3. ✅ **SelectionModel mejorado** - Devuelve filas completas del DataFrame original

---


## 📦 Instalación e Importación


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from BESTLIB.reactive import ReactiveMatrixLayout, SelectionModel
from BESTLIB.matrix import MatrixLayout

print("✅ Librerías importadas correctamente")
print("\n💡 NOTA: Las LinkedViews ahora funcionan dentro de la matriz ASCII")
print("   - No hay duplicación de gráficos")
print("   - El bar chart se actualiza automáticamente usando JavaScript")
print("   - Solo llama display() UNA VEZ para evitar duplicación")


## 🎯 Ejemplo 1: Uso Básico con DataFrame

**Antes:** Tenías que renombrar columnas o crear estructura específica  
**Ahora:** Solo pasas el DataFrame y especificas las columnas


In [ ]:
# Crear DataFrame de ejemplo con nombres de columnas reales
np.random.seed(42)
df = pd.DataFrame({
    'edad': np.random.randint(20, 60, 50),
    'salario': np.random.randint(3000, 15000, 50),
    'departamento': np.random.choice(['IT', 'HR', 'Finance', 'Sales'], 50),
    'años_experiencia': np.random.randint(1, 20, 50),
    'nombre': [f'Empleado_{i}' for i in range(50)]
})

print(f"✅ DataFrame creado con {len(df)} filas")
print("\nPrimeras 5 filas:")
print(df.head())


In [ ]:
# ✨ NUEVO: Usar map_scatter con DataFrame y especificar columnas directamente
MatrixLayout.map_scatter(
    'S',
    df,  # DataFrame directamente
    x_col='edad',           # ← Especificar columna X
    y_col='salario',        # ← Especificar columna Y
    category_col='departamento',  # ← Especificar columna categoría
    interactive=True,
    colorMap={
        'IT': '#e74c3c',
        'HR': '#3498db',
        'Finance': '#2ecc71',
        'Sales': '#f39c12'
    },
    pointRadius=5,
    axes=True
)

layout = MatrixLayout("S")

# Variable para almacenar selección
selected_data = []

def on_select(payload):
    global selected_data
    selected_data = payload.get('items', [])
    print(f"\n✅ {len(selected_data)} filas seleccionadas")
    if selected_data:
        print("\nPrimera fila seleccionada (con TODAS las columnas):")
        print(selected_data[0])

layout.on('select', on_select)
layout.display()


### 🔍 Verificar que tenemos filas completas


In [ ]:
# Selecciona algunos puntos en el gráfico arriba arrastrando el mouse
# Luego ejecuta esta celda

if selected_data:
    print(f"📊 Total de filas seleccionadas: {len(selected_data)}")
    print("\n✅ Verificando que tenemos TODAS las columnas:")
    print(f"  - Columnas disponibles: {list(selected_data[0].keys())}")
    
    # Crear DataFrame con datos seleccionados
    df_selected = pd.DataFrame(selected_data)
    print("\n📋 DataFrame con filas seleccionadas:")
    print(df_selected.head())
    
    print("\n📈 Estadísticas de los datos seleccionados:")
    print(df_selected[['edad', 'salario', 'años_experiencia']].describe())
else:
    print("⚠️ Selecciona algunos puntos en el gráfico arriba arrastrando el mouse")


## 🎯 Ejemplo 2: ReactiveMatrixLayout con LinkedViews Integrado

**Antes:** Tenías que usar LinkedViews separado que creaba gráficos fuera de la matriz  
**Ahora:** Todo está integrado dentro de la matriz ASCII


In [ ]:
# Crear modelo de selección reactivo
selection = SelectionModel()

# ✨ NUEVO: Crear layout reactivo con vistas enlazadas DENTRO de la matriz
layout = ReactiveMatrixLayout("""
SSS
BBB
""", selection_model=selection)

# ✨ NUEVO: Agregar scatter plot especificando columnas directamente
layout.add_scatter(
    'S',
    df,  # DataFrame directamente
    x_col='edad',
    y_col='salario',
    category_col='departamento',
    interactive=True,  # Habilita brush selection
    colorMap={
        'IT': '#e74c3c',
        'HR': '#3498db',
        'Finance': '#2ecc71',
        'Sales': '#f39c12'
    },
    pointRadius=6,
    axes=True
)

# ✨ NUEVO: Agregar bar chart enlazado (se actualiza automáticamente)
layout.add_barchart(
    'B',
    category_col='departamento',  # Especificar columna directamente
    colorMap={
        'IT': '#e74c3c',
        'HR': '#3498db',
        'Finance': '#2ecc71',
        'Sales': '#f39c12'
    },
    axes=True
)

print("✅ Layout creado con scatter plot y bar chart enlazados")
print("\n💡 Instrucciones:")
print("  1. Arrastra el mouse en el scatter plot para seleccionar puntos")
print("  2. Observa cómo el bar chart se actualiza automáticamente (sin duplicación)")
print("  3. Los datos seleccionados contienen TODAS las columnas del DataFrame")
print("\n⚠️ IMPORTANTE: Solo llama display() UNA VEZ para evitar duplicación")

# Mostrar el layout (solo una vez)
layout.display()


### 🔄 Callback Automático


In [ ]:
# ✨ NUEVO: Callback que se ejecuta automáticamente cuando seleccionas
def on_selection_change(items, count):
    print(f"\n🔄 ACTUALIZACIÓN AUTOMÁTICA!")
    print(f"✅ {count} filas seleccionadas")
    
    if count > 0:
        # items contiene TODAS las columnas del DataFrame original
        df_selected = pd.DataFrame(items)
        
        print("\n📊 Distribución por departamento:")
        dept_counts = df_selected['departamento'].value_counts()
        for dept, count in dept_counts.items():
            print(f"   {dept}: {count} empleados")
        
        print("\n📈 Estadísticas:")
        print(f"   Edad promedio: {df_selected['edad'].mean():.1f} años")
        print(f"   Salario promedio: ${df_selected['salario'].mean():,.0f}")
        print(f"   Experiencia promedio: {df_selected['años_experiencia'].mean():.1f} años")

# Registrar callback
selection.on_change(on_selection_change)

print("✅ Callback registrado. Selecciona puntos en el gráfico arriba para ver actualizaciones automáticas.")


### 📊 Acceder a Datos Seleccionados en Cualquier Momento


In [ ]:
# ✨ NUEVO: Acceder a datos seleccionados en cualquier momento
# No necesitas re-ejecutar celdas después de cada selección

selected_rows = selection.get_items()

if selected_rows:
    print(f"📊 Datos actuales: {len(selected_rows)} filas seleccionadas")
    
    # Crear DataFrame con filas seleccionadas
    df_selected = pd.DataFrame(selected_rows)
    
    print("\n📋 Primeras 5 filas (con TODAS las columnas):")
    print(df_selected.head())
    
    print("\n📈 Análisis completo:")
    print(df_selected.describe())
    
    print("\n🏷️ Distribución por departamento:")
    print(df_selected['departamento'].value_counts())
else:
    print("⚠️ No hay selección aún. Selecciona puntos en el scatter plot arriba.")


## 🎯 Ejemplo 3: Dashboard Completo con Múltiples Vistas


In [ ]:
# Crear datos más complejos
np.random.seed(123)
df_completo = pd.DataFrame({
    'ventas_mes': np.random.randint(1000, 10000, 100),
    'gastos_mes': np.random.randint(500, 5000, 100),
    'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 100),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 100),
    'mes': np.random.choice(['Ene', 'Feb', 'Mar', 'Abr'], 100),
    'vendedor_id': np.random.randint(1, 10, 100)
})

# Calcular ganancia
df_completo['ganancia'] = df_completo['ventas_mes'] - df_completo['gastos_mes']

print(f"✅ DataFrame completo creado: {len(df_completo)} filas")
print("\nColumnas:", list(df_completo.columns))
print("\nPrimeras 5 filas:")
print(df_completo.head())


In [ ]:
# Crear dashboard completo con vistas enlazadas
selection_dashboard = SelectionModel()

dashboard = ReactiveMatrixLayout("""
SSSS
BBBB
""", selection_model=selection_dashboard)

# Scatter: Ventas vs Gastos
dashboard.add_scatter(
    'S',
    df_completo,
    x_col='ventas_mes',
    y_col='gastos_mes',
    category_col='region',
    interactive=True,
    colorMap={
        'Norte': '#e74c3c',
        'Sur': '#3498db',
        'Este': '#2ecc71',
        'Oeste': '#f39c12'
    },
    pointRadius=6,
    axes=True
)

# Bar chart: Distribución por región (se actualiza automáticamente)
dashboard.add_barchart(
    'B',
    category_col='region',
    colorMap={
        'Norte': '#e74c3c',
        'Sur': '#3498db',
        'Este': '#2ecc71',
        'Oeste': '#f39c12'
    },
    axes=True
)

# Callback para análisis completo
def on_dashboard_select(items, count):
    if count > 0:
        df_sel = pd.DataFrame(items)
        print(f"\n📊 {count} registros seleccionados")
        print(f"💰 Ganancia total: ${df_sel['ganancia'].sum():,.0f}")
        print(f"📈 Ventas promedio: ${df_sel['ventas_mes'].mean():,.0f}")
        print(f"📉 Gastos promedio: ${df_sel['gastos_mes'].mean():,.0f}")

selection_dashboard.on_change(on_dashboard_select)

print("✅ Dashboard completo creado")
print("\n💡 Selecciona puntos en el scatter plot y observa:")
print("   - El bar chart se actualiza automáticamente (sin duplicación)")
print("   - Tienes acceso a TODAS las columnas del DataFrame")
print("\n⚠️ IMPORTANTE: Solo llama display() UNA VEZ")

# Mostrar el dashboard (solo una vez)
dashboard.display()


### 📈 Análisis Avanzado con Datos Seleccionados


In [ ]:
# Análisis completo de los datos seleccionados
selected = selection_dashboard.get_items()

if selected:
    df_analysis = pd.DataFrame(selected)
    
    print("📊 ANÁLISIS COMPLETO DE DATOS SELECCIONADOS\n")
    
    print("1️⃣ Resumen Estadístico:")
    print(df_analysis[['ventas_mes', 'gastos_mes', 'ganancia']].describe())
    
    print("\n2️⃣ Distribución por Región:")
    print(df_analysis['region'].value_counts())
    
    print("\n3️⃣ Distribución por Producto:")
    print(df_analysis['producto'].value_counts())
    
    print("\n4️⃣ Top 5 Vendedores:")
    top_vendedores = df_analysis.groupby('vendedor_id')['ganancia'].sum().sort_values(ascending=False).head()
    print(top_vendedores)
    
    print("\n✅ Recuerda: Tienes acceso a TODAS las columnas del DataFrame original")
    print(f"   Columnas disponibles: {list(df_analysis.columns)}")
else:
    print("⚠️ Selecciona datos en el dashboard arriba")


## ⚠️ Nota Importante sobre Duplicación

**Si ves gráficos duplicados:**
- Asegúrate de llamar `display()` solo **UNA VEZ** después de configurar todo
- No ejecutes la celda de `display()` múltiples veces
- El bar chart se actualiza automáticamente cuando seleccionas en el scatter plot
- No necesitas re-ejecutar `display()` después de cada selección

**Ejemplo correcto:**
```python
layout = ReactiveMatrixLayout("SB", selection_model=selection)
layout.add_scatter('S', df, ...)
layout.add_barchart('B', ...)
layout.display()  # ← Solo una vez
```

**Ejemplo incorrecto (causa duplicación):**
```python
layout.display()  # Primera vez
layout.display()  # ❌ Segunda vez - causa duplicación
```


## 🎯 Ejemplo 4: Bar Chart con Agregación


In [ ]:
# Crear datos de ventas por producto
df_ventas = pd.DataFrame({
    'producto': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B'],
    'ventas': [1000, 2000, 1500, 1200, 1800, 1600, 1100, 1900],
    'region': ['Norte', 'Sur', 'Este', 'Norte', 'Sur', 'Este', 'Norte', 'Sur']
})

print("📋 Datos de ventas:")
print(df_ventas)

# ✨ NUEVO: Bar chart con agregación (suma de ventas por producto)
MatrixLayout.map_barchart(
    'B',
    df_ventas,
    category_col='producto',  # Agrupar por producto
    value_col='ventas',       # Sumar ventas
    color='#3498db',
    interactive=True,
    axes=True
)

layout_ventas = MatrixLayout("B")

def on_select_ventas(payload):
    items = payload.get('items', [])
    if items:
        print(f"\n✅ {len(items)} filas seleccionadas")
        print("\n📋 Filas completas (con todas las columnas):")
        df_sel = pd.DataFrame(items)
        print(df_sel)

layout_ventas.on('select', on_select_ventas)
print("\n✅ Bar chart con agregación creado")
print("💡 Selecciona barras para ver las filas originales completas")
layout_ventas.display()


## 📝 Resumen de Mejoras

### ✅ 1. Soporte para DataFrames
- **Antes:** Tenías que renombrar columnas o crear estructura específica
- **Ahora:** Solo pasas el DataFrame y especificas columnas con `x_col`, `y_col`, `category_col`

### ✅ 2. LinkedViews Integrado
- **Antes:** LinkedViews creaba gráficos fuera de la matriz
- **Ahora:** Todo está dentro de `ReactiveMatrixLayout` usando el layout ASCII

### ✅ 3. Filas Completas
- **Antes:** Solo obtenías datos del gráfico (`x`, `y`, `category`)
- **Ahora:** Obtienes TODAS las columnas del DataFrame original

---

**¡Disfruta de las nuevas funcionalidades! 🎉**
